In [0]:
pip install openpyxl

In [0]:
%restart_python

# Bronze tables
We load here the data as it is, we identify the variables with unique values to set as index

https://docs.databricks.com/aws/en/tables/managed

We choose Fully managed & Databricks-native tables using Managed Delta table

In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name
import pandas as pd

spark = SparkSession.builder.getOrCreate()
# we must allows to overwrite the schema because dataset are evolving (paid version)
# spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")


### Create Schema if not exists

In [0]:
CATALOG_NAME = "databricks_hackathon"
SCHEMA_NAME = "bronze"
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}")
spark.sql(f"USE {CATALOG_NAME}.{SCHEMA_NAME}")

### helper functions to load different type of data

In [0]:
from pyspark.sql.functions import current_timestamp, lit

def load_lucillus_data(batch_name,pdf):
    print(f"load_lucillus_data {batch_name} ...")
    df = pdf['Lucullus Data'].copy().iloc[5:,:]
    df.columns = df.columns.str.strip().str.lower()
    # we need rename to keep the unit of columns
    new_columns = [f"{a}_{b}".strip().lower() if str(b).lower() != 'nan' else a for a, b in zip(pdf['Lucullus Data'].columns, pdf['Lucullus Data'].iloc[0,:])]
    df.columns = new_columns
    # replace special caracter that caus eproblem with spark indexing
    df.columns = df.columns.str.replace(r'\s','',regex=True).str.replace(r'[ °,;{}/\(\)\n\t=]', '_', regex=True).str.replace("%","percent")
    df.Timestamp = pd.to_datetime(df.Timestamp.copy(), dayfirst=True)
    # remove completelly empty columns (else spak can't assign type and raise error VOID)
    df = df.dropna(axis=1, how='all')
    # Convert to Spark DataFrame
    sdf = spark.createDataFrame(df)

    # Add metadata columns
    sdf = (sdf
        .withColumn("ingestion_time", current_timestamp())
        .withColumn("project", lit("hackathon"))
        .withColumn("source_file", lit(raw_excel_path))
        .withColumn("batch_id", lit(batch_name))
    )

    return sdf

In [0]:
def load_capacitance_data(batch_name,pdf):
    print(f"load_capacitance_data{batch_name} ...")
    # add capacitance data 
    table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.Capacitance"

    
    if pdf['Capacitance'].copy().shape[1] != 21: #. <-- 3 detaset with different shape MAKE 
        pdf_cap = pdf['Capacitance'].copy().iloc[:, :21]
    else:
        pdf_cap = pdf['Capacitance'].copy()

    # make test to attribute columns if data distribution matches




    pdf_cap.columns = pdf_cap.columns.str.strip().str.lower()
    # replace special caracter that caus eproblem with spark indexing
    pdf_cap.columns = pdf_cap.columns.str.replace(r'\s','',regex=True)\
                                    .str.replace(r'[ ,;{}/\(\)\n\t=%]', '_', regex=True)

    cols = [(c,pdf_cap[c].nunique()) for c in pdf_cap.columns]
    cols_to_keep_as_index = [c[0] for c in cols if c[1] == 1]
    pdf_ok = pdf_cap.copy().loc[:,~pdf_cap.columns.isin(cols_to_keep_as_index)]

    cap_sdf = spark.createDataFrame(pdf_ok)
    # add ingestion metadata
    cap_sdf = (cap_sdf
        .withColumn("ingestion_time", current_timestamp())
        .withColumn("project", lit("hackathon"))
        .withColumn("source_file", lit(raw_excel_path))
        .withColumn("batch_id", lit(batch_name))
    )
    # add metadata as index ==> could create a new bronze_metadata table to store it more efficiently
    for c in cols_to_keep_as_index:
        cap_sdf = (cap_sdf
        .withColumn(c, lit(pdf_cap[c].unique()[0]))
    )

    return cap_sdf


In [0]:
def create_or_merge_table(sdf,table_name):
    print(f'merging OPT {table_name} ...')

    # Quoted FQN for SQL
    table_fqn = ".".join([f"`{a}`" for a in table_name.split(".")])

    # Define the MERGE SQL query based on the table type
    if table_name.startswith("timeseries"):
        merge_sql = f"""
            MERGE INTO {table_fqn} AS target
            USING updates
            ON
                target.timestamp = updates.timestamp AND
                target.batch_id = updates.batch_id AND
                target.source_file = updates.source_file
            WHEN NOT MATCHED THEN INSERT *
        """
    elif table_name.startswith("capacitance"):
        merge_sql = f"""
            MERGE INTO {table_fqn} AS target
            USING updates
            ON
                target.time_stamp = updates.time_stamp AND
                target.batch_id = updates.batch_id AND
                target.source_file = updates.source_file
            WHEN NOT MATCHED THEN INSERT *
        """
    if spark.catalog.tableExists(table_name):
        # Register the source DataFrame as a temporary view
        sdf.createOrReplaceTempView("updates")

        # Execute the MERGE using Spark SQL
        spark.sql(merge_sql)
        print(f"Data MERGED (upserted) into existing table: {table_name}. Duplicates avoided.")
    else:
        # Create a new table with schema evolution enabled
        sdf.write \
            .format("delta") \
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .saveAsTable(table_name)
        print(f"New table created: {table_name}")



In [0]:
def create_or_merge_table_OLD(sdf,table_name):
    from delta.tables import DeltaTable
    from pyspark.sql.functions import current_timestamp, lit
    print(f'merging {table_name} ...')

    # Quoted FQN for SQL
    table_fqn = ".".join([f"`{a}`" for a in table_name.split(".")])
    # Define the unique merge key to avoid duplicates upon re-running script
    if table_name.startswith("timeseries"):
        merge_condition = f"""
            target.timestamp = updates.timestamp AND 
            target.batch_id = updates.batch_id AND
            target.source_file = updates.source_file
        """
    elif table_name.startswith("capacitance"):
        merge_condition = f"""
            target.time_stamp = updates.time_stamp AND 
            target.batch_id = updates.batch_id AND
            target.source_file = updates.source_file
        """

    
    if spark.catalog.tableExists(table_name):
        # The table already exists - perform a MERGE
        
        deltaTable = DeltaTable.forName(spark, table_fqn)
        
        deltaTable.alias("target") \
            .merge(
                source=sdf.alias(f"updates"),
                condition=merge_condition
            ) \
            .whenNotMatchedInsertAll() \
            .execute()
        
        print(f"Data MERGED (upserted) into existing table: {table_name}. Duplicates avoided.")
        
    else: # enable option merge for evolution of schemas
        sdf.write \
            .format("delta") \
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .saveAsTable(table_name)
        print(f"New table created: {table_name}")

    print("End load_capacitance_data")

### iterate over files and load data to delta lake

In [0]:
import os
import pandas as pd
from pyspark.sql.functions import current_timestamp, lit

raw_excel_path = "/Volumes/databricks_hackathon/lucullus_data/lucullus_data_raw/"

excel_files = [f for f in os.listdir(raw_excel_path) if f.endswith(".xlsx") and "Lucullus" in f]

for f in excel_files:
    if  "CellGrowth" in f:
        continue
    batch_name = f.split("_")[0]
    print(f"Processing {batch_name}...")
    
    pdf = pd.read_excel(raw_excel_path + f, sheet_name=None)

    try:
        print("...")
        sdf_1 = load_lucillus_data(batch_name,pdf)
        table_name = f"timeseries_bronze_{batch_name}"
        create_or_merge_table(sdf_1,table_name)

    except Exception as e:
        print("There was an error to load_lucillus_data: ",e)

    try:
        print("...")
        sdf_2 = load_capacitance_data(batch_name,pdf)
        table_name = f"capacitance_bronze_{batch_name}"
        create_or_merge_table(sdf_2,table_name)

    except Exception as e:
        print("There was an error to load_capacitance_data: ",e)
    



### verify that tables were created

In [0]:
def check_table_exist(table_name):
    try:
        df_check = spark.read.table(table_name)
        row_count = df_check.count()
        print(f"Verification successful: The table '{table_name}' is readable.")
        print(f"Total rows found: {row_count}")
    except:
        print(f"* * * Table '{table_name}' does not exist.")


for f in excel_files:
    try:
        batch_name = f.split("_")[0]

        table_name = f"Timeseries_bronze_{batch_name}"
        check_table_exist(table_name)

        table_name = f"Capacitance_bronze_{batch_name}"
        check_table_exist(table_name)
        
    except Exception as e :
        print("{*} table not exists: ", e)

### reset schemas

In [0]:
#spark.sql(f"DROP SCHEMA IF EXISTS {CATALOG_NAME}.bronze CASCADE")
#spark.sql(f"DROP SCHEMA IF EXISTS {CATALOG_NAME}.silver CASCADE")
#spark.sql(f"DROP SCHEMA IF EXISTS {CATALOG_NAME}.gold CASCADE")